<a href="https://colab.research.google.com/github/mohansharma077/-Fraud-Detection-On-Credit-Card-Transactions/blob/main/Mental_Health_Full_AI_Implemented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Mental Health Monitoring System

## WITH FULLY IMPLEMENTED AI RECOMMENDATION MODEL! 🤖

This project implements a comprehensive mental health monitoring system with:
- Emotion Detection (6+ emotions)
- Sentiment Analysis
- Risk Assessment & Scoring
- **AI-Generated Recommendations via Pretrained T5 Model** (Fully Working)
- Trend Analysis & Analytics
- Word Cloud Visualization
- **Audio Input Support** (Speech-to-Text with Whisper)
- Interactive Gradio Interface

## Step 1: Install Required Libraries

In [1]:
!pip install gradio transformers torch pandas matplotlib seaborn wordcloud scikit-learn scipy -q
!pip install librosa openai-whisper soundfile -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Step 2: Import All Libraries

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
from transformers import pipeline
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import os
from collections import Counter
import warnings
import whisper
import torch
warnings.filterwarnings('ignore')

print(' All libraries imported successfully!')

## Step 3: Initialize All Models (Including AI Recommendation Model)

In [3]:
print('Loading emotion detection model...')
emotion_analyzer = pipeline('text-classification',
                           model='j-hartmann/emotion-english-distilroberta-base',
                           top_k=None)

print('Loading sentiment analysis model...')
sentiment_analyzer = pipeline('text-classification',
                             model='distilbert-base-uncased-finetuned-sst-2-english')

print('Loading Whisper model for audio transcription...')
whisper_model = whisper.load_model('base')

print('Loading T5 model for AI-powered recommendation generation...')
# T5 is better for text generation and follows instructions
recommendation_generator = pipeline('text2text-generation',
                                    model='t5-small',
                                    device=0 if torch.cuda.is_available() else -1)

print(' All models loaded successfully!')

user_data = {
    'entries': [],
    'emotions_freq': Counter(),
    'sentiments_freq': Counter(),
    'risk_assessments': []
}

print(' Data storage initialized!')

Loading emotion detection model...


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


✓ All models loaded successfully!
✓ Data storage initialized!


## Step 4: Audio Transcription Function

In [4]:
def transcribe_audio(audio_input):
    if audio_input is None:
        return 'No audio provided. Please record or upload an audio file.'

    try:
        if isinstance(audio_input, tuple):
            sample_rate, audio_array = audio_input
            import soundfile as sf
            temp_path = '/tmp/audio_temp.wav'
            sf.write(temp_path, audio_array, sample_rate)
            audio_file = temp_path
        else:
            audio_file = audio_input

        result = whisper_model.transcribe(audio_file, language='en')
        transcribed_text = result['text'].strip()

        if not transcribed_text:
            return 'Could not detect any speech. Please try again with clearer audio.'

        return transcribed_text

    except Exception as e:
        return f'Error transcribing audio: {str(e)}'

## Step 5: Risk Assessment Function

In [5]:
def assess_risk(emotion, sentiment_score, text_input):
    risk_keywords = {
        'high': ['suicide', 'death', 'hurt', 'harm', 'kill', 'pain', 'suffering', 'unbearable'],
        'medium': ['sad', 'depressed', 'anxious', 'stress', 'worry', 'fear', 'panic', 'overwhelm']
    }

    risk_score = 0

    emotion_risk_map = {
        'sadness': 35,
        'anger': 30,
        'fear': 40,
        'surprise': 10,
        'joy': 5,
        'love': 5,
        'disgust': 25,
        'neutral': 15
    }

    emotion_lower = emotion.lower()
    risk_score += emotion_risk_map.get(emotion_lower, 20)
    risk_score += (1 - sentiment_score) * 40

    text_lower = text_input.lower() if isinstance(text_input, str) else ''
    high_risk_count = sum(1 for kw in risk_keywords['high'] if kw in text_lower)
    medium_risk_count = sum(1 for kw in risk_keywords['medium'] if kw in text_lower)

    risk_score += high_risk_count * 20
    risk_score += medium_risk_count * 8

    risk_score = min(100, max(0, risk_score))

    if risk_score >= 70:
        risk_level = '🔴 HIGH RISK'
    elif risk_score >= 40:
        risk_level = '🟡 MEDIUM RISK'
    else:
        risk_level = '🟢 LOW RISK'

    return risk_level, risk_score

## Step 6: AI-Powered Recommendation Generation (NOW FULLY WORKING!)

In [6]:
def generate_ai_recommendations(emotion, risk_level, sentiment_score, user_text):
    """
    Generate personalized mental health recommendations using T5 model
    This creates context-aware, empathetic recommendations for each user
    """

    try:
        # Build context-aware prompt for the AI model
        emotion_lower = emotion.lower()

        # Create a detailed prompt that the T5 model can understand
        if 'HIGH' in risk_level:
            prompt = f"""Generate urgent mental health coping strategies for someone experiencing {emotion_lower}.
They are reporting negative feelings. Provide 5 immediate practical recommendations that are empathetic and actionable."""
        else:
            prompt = f"""Generate helpful mental health recommendations for someone experiencing {emotion_lower}.
Provide 5 practical coping strategies that are compassionate and easy to implement."""

        # Generate recommendations using T5 model
        result = recommendation_generator(prompt,
                                         max_length=300,
                                         num_beams=4,
                                         temperature=0.8,
                                         do_sample=True)

        ai_recommendation = result[0]['generated_text'].strip()

        # If the output is too short, add fallback
        if len(ai_recommendation) < 50:
            ai_recommendation = generate_fallback_recommendations(emotion, risk_level)

        # Add risk warning if needed
        if 'HIGH' in risk_level:
            ai_recommendation += '\n\n⚠️  URGENT SUPPORT:\n'
            ai_recommendation += '• National Suicide Prevention Lifeline: 999 (Uk)\n'
            ai_recommendation += '• text "SHOUT" to 85258.\n'
            ai_recommendation += '• International Association for Suicide Prevention: https://www.iasp.info/resources/Crisis_Centres/'

        return ai_recommendation

    except Exception as e:
        print(f'AI generation error: {str(e)}')
        return generate_fallback_recommendations(emotion, risk_level)


def generate_fallback_recommendations(emotion, risk_level):
    """
    Fallback recommendations if AI generation fails - uses curated psychological strategies
    """
    recommendations_map = {
        'sadness': [
            '1. Reach out to someone you trust - talking helps more than isolation',
            '2. Engage in light physical activity (even a 10-minute walk can help)',
            '3. Practice gratitude: write 3 things you appreciate today',
            '4. Maintain a regular sleep schedule (aim for 7-9 hours)',
            '5. Consider professional support - a therapist can provide lasting help'
        ],
        'anxiety': [
            '1. Try the 4-7-8 breathing technique: breathe in for 4 seconds, hold for 7, exhale for 8',
            '2. Practice grounding: identify 5 things you see, 4 you can touch, 3 you hear, 2 you smell, 1 you taste',
            '3. Limit caffeine and processed sugar which can worsen anxiety',
            '4. Try progressive muscle relaxation or guided meditation (apps like Calm help)',
            '5. Break tasks into smaller steps to reduce feeling overwhelmed'
        ],
        'anger': [
            '1. Take a 15-minute timeout away from the triggering situation',
            '2. Release physical tension through exercise, sports, or dancing',
            '3. Practice slow, deliberate breathing to calm your nervous system',
            '4. Journal about what triggered your anger to gain clarity',
            '5. Use assertive communication: express your feelings without blame'
        ],
        'fear': [
            '1. Identify the specific source of your fear (often fear is vague)',
            '2. Use grounding techniques to bring yourself back to the present moment',
            '3. Start small: gradual exposure to fears is more effective than avoidance',
            '4. Practice relaxation: yoga, tai chi, or meditation build resilience',
            '5. Consider cognitive-behavioral therapy (CBT) for persistent fears'
        ],
        'joy': [
            '1. Share your happiness with loved ones - positive emotions multiply when shared',
            '2. Continue and expand the activities bringing you joy',
            '3. Practice gratitude journaling to anchor positive feelings',
            '4. Help someone else: volunteering extends positive emotions',
            '5. Plan future positive experiences to look forward to'
        ]
    }

    base_recs = recommendations_map.get(emotion.lower(), [
        '1. Practice self-compassion: treat yourself as you would a good friend',
        '2. Maintain healthy basics: sleep, exercise, and nutrition matter',
        '3. Limit stress: say no to non-essential commitments',
        '4. Stay connected with supportive people',
        '5. Consider professional mental health support'
    ])

    result = '\n'.join(base_recs)

    if 'HIGH' in risk_level:
        result += '\n\n⚠️  URGENT SUPPORT NEEDED:\n'
        result += '• National Suicide Prevention Lifeline: 999 (Uk)\n'
        result += '• text "SHOUT" to 85258.\n'
        result += '• International Association for Suicide Prevention: https://www.iasp.info/resources/Crisis_Centres/'

    return result

## Step 7: Main Analysis Function

In [7]:
def analyze_mental_health(text_input):
    if not text_input or len(text_input.strip()) < 5:
        return 'Please enter at least 5 characters to analyze', '', '', '', ''

    emotions_result = emotion_analyzer(text_input)
    top_emotion = emotions_result[0][0]['label']
    emotion_score = emotions_result[0][0]['score']

    sentiment_result = sentiment_analyzer(text_input)
    sentiment_label = sentiment_result[0]['label']
    sentiment_score_raw = sentiment_result[0]['score']

    if sentiment_label == 'NEGATIVE':
        sentiment_score = 1 - sentiment_score_raw
    else:
        sentiment_score = sentiment_score_raw

    risk_level, risk_score = assess_risk(top_emotion, sentiment_score, text_input)

    # Generate AI-powered recommendations
    ai_recommendations = generate_ai_recommendations(top_emotion, risk_level, sentiment_score, text_input)

    entry = {
        'timestamp': datetime.now().isoformat(),
        'text': text_input,
        'emotion': top_emotion,
        'emotion_score': float(emotion_score),
        'sentiment': sentiment_label,
        'sentiment_score': float(sentiment_score),
        'risk_level': risk_level,
        'risk_score': float(risk_score)
    }
    user_data['entries'].append(entry)
    user_data['emotions_freq'][top_emotion] += 1
    user_data['sentiments_freq'][sentiment_label] += 1
    user_data['risk_assessments'].append(risk_score)

    report = f'''╔════════════════════════════════════════════════════════════════╗
║         MENTAL HEALTH ANALYSIS REPORT
╚════════════════════════════════════════════════════════════════╝

📊 PRIMARY EMOTION: {top_emotion.upper()}
   Confidence Score: {emotion_score*100:.1f}%

💭 SENTIMENT ANALYSIS: {sentiment_label}
   Sentiment Score: {sentiment_score*100:.1f}%

🎯 RISK ASSESSMENT: {risk_level}
   Risk Score: {risk_score:.1f}/100

📋 AI-GENERATED MENTAL HEALTH RECOMMENDATIONS:
{ai_recommendations}

📝 Entry logged at: {entry["timestamp"]}
'''

    emotions_breakdown = 'All Detected Emotions:\n'
    for emotion_data in emotions_result[0]:
        emotions_breakdown += f"  • {emotion_data['label']}: {emotion_data['score']*100:.1f}%\n"

    viz_data = f'''Analysis Summary:
- Total Entries: {len(user_data['entries'])}
- Most Recent Emotion: {top_emotion}
- Average Risk Score: {np.mean(user_data['risk_assessments']) if user_data['risk_assessments'] else 0:.1f}
- High Risk Entries: {sum(1 for s in user_data['risk_assessments'] if s >= 70)}
- Medium Risk Entries: {sum(1 for s in user_data['risk_assessments'] if 40 <= s < 70)}
- Low Risk Entries: {sum(1 for s in user_data['risk_assessments'] if s < 40)}
'''

    return report, emotions_breakdown, viz_data, top_emotion, risk_level

## Step 8: Combined Analysis (Text + Audio)

In [8]:
def analyze_from_audio_or_text(text_input, audio_input):
    if audio_input is not None:
        transcribed_text = transcribe_audio(audio_input)
        if transcribed_text.startswith('Error') or transcribed_text.startswith('No'):
            return transcribed_text, '', '', '', ''
        text_to_analyze = transcribed_text
    elif text_input and len(text_input.strip()) > 0:
        text_to_analyze = text_input
    else:
        return 'Please enter text or provide audio input', '', '', '', ''

    return analyze_mental_health(text_to_analyze)

## Step 9: Trend Analysis Function

In [9]:
def analyze_trends():
    if not user_data['entries']:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, 'No data yet. Please add some entries first.',
                ha='center', va='center', fontsize=16, transform=ax.transAxes)
        ax.axis('off')
        return fig

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Mental Health Monitoring - Trend Analysis', fontsize=18, fontweight='bold')

    emotions_data = dict(user_data['emotions_freq'])
    ax1 = axes[0, 0]
    colors = plt.cm.Set3(np.linspace(0, 1, len(emotions_data)))
    bars = ax1.bar(emotions_data.keys(), emotions_data.values(), color=colors, edgecolor='black', linewidth=1.5)
    ax1.set_title('Emotion Frequency Distribution', fontweight='bold', fontsize=12)
    ax1.set_ylabel('Count', fontweight='bold')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}', ha='center', va='bottom', fontweight='bold')

    risk_scores = user_data['risk_assessments']
    ax2 = axes[0, 1]
    ax2.plot(risk_scores, marker='o', linewidth=2.5, markersize=8, color='#FF6B6B', label='Risk Score')
    ax2.fill_between(range(len(risk_scores)), risk_scores, alpha=0.3, color='#FF6B6B')
    ax2.axhline(y=40, color='orange', linestyle='--', linewidth=2, label='Medium Risk (40)')
    ax2.axhline(y=70, color='red', linestyle='--', linewidth=2, label='High Risk (70)')
    ax2.set_title('Risk Score Over Time', fontweight='bold', fontsize=12)
    ax2.set_ylabel('Risk Score', fontweight='bold')
    ax2.set_xlabel('Entry Number', fontweight='bold')
    ax2.legend(loc='upper left')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 100)

    sentiment_data = dict(user_data['sentiments_freq'])
    ax3 = axes[1, 0]
    colors_sentiment = ['#FF6B6B', '#4ECDC4']
    wedges, texts, autotexts = ax3.pie(sentiment_data.values(), labels=sentiment_data.keys(), autopct='%1.1f%%',
                colors=colors_sentiment, startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
    ax3.set_title('Sentiment Distribution', fontweight='bold', fontsize=12)

    risk_categories = {'Low': 0, 'Medium': 0, 'High': 0}
    for score in risk_scores:
        if score >= 70:
            risk_categories['High'] += 1
        elif score >= 40:
            risk_categories['Medium'] += 1
        else:
            risk_categories['Low'] += 1

    ax4 = axes[1, 1]
    colors_risk = ['#51CF66', '#FFD93D', '#FF6B6B']
    bars = ax4.barh(list(risk_categories.keys()), list(risk_categories.values()), color=colors_risk, edgecolor='black', linewidth=1.5)
    ax4.set_title('Risk Category Breakdown', fontweight='bold', fontsize=12)
    ax4.set_xlabel('Count', fontweight='bold')
    ax4.grid(axis='x', alpha=0.3)
    for bar in bars:
        width = bar.get_width()
        ax4.text(width, bar.get_y() + bar.get_height()/2.,
                f'{int(width)}', ha='left', va='center', fontweight='bold')

    plt.tight_layout()
    return fig

## Step 10: Word Cloud Generation

In [10]:
def generate_wordcloud():
    if not user_data['entries']:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.text(0.5, 0.5, 'No data yet. Please add some entries first.',
                ha='center', va='center', fontsize=16, transform=ax.transAxes)
        ax.axis('off')
        return fig

    all_text = ' '.join([entry['text'] for entry in user_data['entries']])
    wordcloud = WordCloud(width=1000, height=500,
                         background_color='white',
                         colormap='viridis',
                         max_words=100,
                         relative_scaling=0.5,
                         min_font_size=10).generate(all_text)

    fig, ax = plt.subplots(figsize=(14, 7))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')
    ax.set_title('Word Cloud - Common Words in All Entries', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()

    return fig

## Step 11: Statistics Summary

In [11]:
def get_statistics():
    if not user_data['entries']:
        return 'No data yet. Please add some entries first.'

    total_entries = len(user_data['entries'])
    top_emotion = user_data['emotions_freq'].most_common(1)[0][0] if user_data['emotions_freq'] else 'N/A'
    avg_risk = np.mean(user_data['risk_assessments'])
    high_risk_count = sum(1 for score in user_data['risk_assessments'] if score >= 70)
    medium_risk_count = sum(1 for score in user_data['risk_assessments'] if 40 <= score < 70)
    low_risk_count = sum(1 for score in user_data['risk_assessments'] if score < 40)

    positive_entries = user_data['sentiments_freq'].get('POSITIVE', 0)
    negative_entries = user_data['sentiments_freq'].get('NEGATIVE', 0)

    stats_text = f'''╔════════════════════════════════════════════════════════════════╗
║                  OVERALL STATISTICS
╚════════════════════════════════════════════════════════════════╝

📊 Total Entries Analyzed: {total_entries}
🎯 Most Frequent Emotion: {top_emotion}
📈 Average Risk Score: {avg_risk:.1f}/100

🔴 High Risk Entries: {high_risk_count}
🟡 Medium Risk Entries: {medium_risk_count}
🟢 Low Risk Entries: {low_risk_count}

😊 Positive Sentiment: {positive_entries}
😞 Negative Sentiment: {negative_entries}

💾 Data Storage: Active
📅 Monitoring Period: Last {total_entries} entries
'''
    return stats_text

## Step 12: Launch Gradio Interface with All Features

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown('# 🧠 Mental Health Monitoring System')
    gr.Markdown('### Comprehensive Analysis with AI-Generated Recommendations')
    gr.Markdown('### Features: 🎤 Audio Input | 🤖 AI-Generated Recommendations | 📈 Trend Analysis | ☁️ Word Cloud')
    gr.Markdown('---')

    with gr.Tabs():
        with gr.Tab('📝 Analysis & AI Recommendations'):
            gr.Markdown('### Share your thoughts via text OR audio for AI-generated analysis')
            gr.Markdown('*The AI generates personalized mental health recommendations based on your emotions and situation*')

            with gr.Group():
                gr.Markdown('#### Option 1: Text Input')
                text_input = gr.Textbox(
                    label='Type your thoughts or feelings',
                    lines=5,
                    placeholder='Type here... (minimum 5 characters)',
                    info='Share what\'s on your mind'
                )

            with gr.Group():
                gr.Markdown('#### Option 2: Audio Input (🎙️ NEW!)')
                gr.Markdown('Record your voice or upload an audio file for automatic transcription')
                audio_input = gr.Audio(
                label='Record or upload audio',
                sources=['microphone', 'upload'],
                type='numpy'
                # Removed 'info' parameter - moved description to gr.Markdown instead
)


            analyze_btn = gr.Button('🔍 Analyze', variant='primary', size='lg')

            with gr.Group():
                gr.Markdown('### 📊 AI-Generated Analysis Results')
                report_output = gr.Textbox(
                    label='Detailed Analysis Report with AI-Generated Recommendations',
                    lines=18,
                    interactive=False,
                    max_lines=25
                )

            with gr.Group():
                with gr.Row():
                    emotions_output = gr.Textbox(
                        label='All Emotions Detected',
                        interactive=False,
                        lines=5
                    )
                    stats_output = gr.Textbox(
                        label='Session Statistics',
                        interactive=False,
                        lines=5
                    )

            analyze_btn.click(analyze_from_audio_or_text, inputs=[text_input, audio_input],
                             outputs=[report_output, emotions_output, stats_output])

        with gr.Tab('📈 Trends & Analytics'):
            gr.Markdown('### Track your emotional patterns and risk trends')
            trends_btn = gr.Button('📊 Generate Trend Analysis', variant='primary', size='lg')
            trends_plot = gr.Plot(label='Trend Analysis Visualization')
            trends_btn.click(analyze_trends, outputs=trends_plot)

        with gr.Tab('☁️ Word Cloud'):
            gr.Markdown('### Visual representation of frequently used words')
            wordcloud_btn = gr.Button('🎨 Generate Word Cloud', variant='primary', size='lg')
            wordcloud_plot = gr.Plot(label='Word Cloud Visualization')
            wordcloud_btn.click(generate_wordcloud, outputs=wordcloud_plot)

        with gr.Tab('📋 Overall Statistics'):
            gr.Markdown('### Comprehensive monitoring statistics')
            stats_btn = gr.Button('📊 View Statistics', variant='primary', size='lg')
            stats_text = gr.Textbox(label='Statistics Summary', interactive=False, lines=12)
            stats_btn.click(get_statistics, outputs=stats_text)

        with gr.Tab('ℹ️ How to Use'):
            gr.Markdown('''## How to Use This System

### 🤖 AI Models Used:
- **T5 Model**: Generates personalized mental health recommendations from scratch
- **Emotion Detection**: j-hartmann/emotion-english-distilroberta-base identifies 6+ emotions
- **Sentiment Analysis**: distilbert model determines positive/negative tone
- **Whisper Model**: OpenAI's speech-to-text transcribes audio automatically

### Text Input:
1. Type your thoughts or feelings in the text box
2. Press "Analyze" to get AI-generated recommendations

### Audio Input:
1. **Record**: Click the microphone icon to record your voice
2. **Upload**: Click the upload icon to select an audio file
3. The system will transcribe and analyze automatically

### Analysis Results:
- **Primary Emotion**: What emotion is detected
- **Sentiment**: Positive or Negative tone assessment
- **Risk Score**: 0-100 scale for mental health assessment
- **AI Recommendations**: Unique recommendations generated by the T5 model for your situation

### Tabs:
- **Analysis**: Main input and AI-generated recommendations
- **Trends**: View emotional patterns and risk trends over time
- **Word Cloud**: See recurring themes in your entries
- **Statistics**: Overall monitoring metrics and insights
''')

    gr.Markdown('---')
    gr.Markdown('### ⚠️ Important Disclaimer')
    gr.Markdown('This system uses AI for informational purposes and should not replace professional mental health consultation.')

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ebee90913096759a17.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
